In [26]:
import mne
import numpy as np
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [3]:
file  ="C:\\Users\\Luchitha\\Documents\\BCI\\BCI_IV\\dataset\\A01T.gdf"

In [4]:
import mne

raw_train = mne.io.read_raw_gdf(file, preload=True)
raw_train.filter(8., 30., fir_design='firwin')


Extracting GDF parameters from C:\Users\Luchitha\Documents\BCI\BCI_IV\dataset\A01T.gdf...
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


C:\Users\Luchitha\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)



<RawGDF | A01T.gdf, 25 x 672528 (2690.1 s), ~128.3 MiB, data loaded>

In [9]:
raw_train.pick_types(eeg=True)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


<RawGDF | A01T.gdf, 25 x 672528 (2690.1 s), ~128.3 MiB, data loaded>

In [10]:
events_train, event_id = mne.events_from_annotations(raw_train)
print(event_id)


Used Annotations descriptions: [np.str_('1023'), np.str_('1072'), np.str_('276'), np.str_('277'), np.str_('32766'), np.str_('768'), np.str_('769'), np.str_('770'), np.str_('771'), np.str_('772')]
{np.str_('1023'): 1, np.str_('1072'): 2, np.str_('276'): 3, np.str_('277'): 4, np.str_('32766'): 5, np.str_('768'): 6, np.str_('769'): 7, np.str_('770'): 8, np.str_('771'): 9, np.str_('772'): 10}


In [11]:
event_dict = {
    '769': event_id['769'],
    '770': event_id['770'],
    '771': event_id['771'],
    '772': event_id['772']
}


In [12]:
epochs_train = mne.Epochs(
    raw_train,
    events_train,
    event_id=event_dict,
    tmin=2.0,
    tmax=6.0,
    baseline=None,
    preload=True
)


Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 1001 original time points ...
1 bad epochs dropped


In [13]:
print(epochs_train.get_data().shape)


(287, 25, 1001)


In [15]:

y_train = epochs_train.events[:, -1]
print(np.unique(y_train, return_counts=True))


(array([ 7,  8,  9, 10]), array([71, 72, 72, 72]))


In [16]:
X_train = epochs_train.get_data()
y_train = epochs_train.events[:, -1]


In [17]:
print(X_train.shape)
print(np.unique(y_train, return_counts=True))


(287, 25, 1001)
(array([ 7,  8,  9, 10]), array([71, 72, 72, 72]))


In [18]:
y_train = y_train - y_train.min()
print(np.unique(y_train, return_counts=True))

(array([0, 1, 2, 3]), array([71, 72, 72, 72]))


In [ ]:

csp = CSP(n_components=4, log=True, norm_trace=False)
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')

clf = Pipeline([
    ('CSP', csp),
    ('LDA', lda)
])

clf.fit(X_train, y_train)

train_acc = clf.score(X_train, y_train)
print("Training Accuracy:", train_acc)


Computing rank from data with rank=None
    Using tolerance 0.00041 (2.2e-16 eps * 25 dim * 7.4e+10  max singular value)
    Estimated rank (data): 25
    data: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Training Accuracy: 0.6236933797909407


In [27]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(clf, X_train, y_train, cv=cv)

print("CV Accuracy:", scores.mean())

Computing rank from data with rank=None
    Using tolerance 0.00032 (2.2e-16 eps * 25 dim * 5.7e+10  max singular value)
    Estimated rank (data): 25
    data: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.00037 (2.2e-16 eps * 25 dim * 6.6e+10  max singular value)
    Estimated rank (data): 25
    data: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.00037 (2.2e-16 eps *